In [0]:
# %run ../00_config

In [0]:
print("フライト予約履歴CSVファイルを作成します...")

In [0]:
from pyspark.sql import Row
from pyspark.sql.functions import col
import random, datetime, math

# ============================================================
# テーブル: flight_booking（フライト予約履歴）サンプル
# ============================================================
# flight_booking  サンプルデータ生成（シンプル版）
#   * 便×出発日ごと     : 500 予約
#   * 会員ごとの搭乗回数: 1–5 回
#   * 総レコード数      : N_ROWS  ※BOOKINGS_PER_PAIR の倍数推奨

# ---------- パラメータ ----------
N_USERS = 3000        # 最初に用意しておく会員数（足りなければ自動で増える）
N_ROWS  = 20000       # 欲しい予約レコード数（500 の倍数にしておくと端数処理不要）
MAX_PER_USER = 5      # 会員あたり最大搭乗回数
MIN_PER_USER = 1      # 会員あたり最小搭乗回数
BOOKINGS_PER_PAIR = 500  # 便×出発日ごとの予約件数

airlines = ["JL", "NH", "GK", "MM", "BC"]
routes   = [("HND", "ITM"), ("HND", "CTS"), ("HND", "SFO"),
            ("LAX", "HND"), ("NRT", "SIN"), ("KIX", "NRT"),
            ("SYD", "HND"), ("CDG", "NRT"), ("HND", "OKA"), ("FUK", "HND")]

start_date = datetime.date(2024, 1, 1)
end_date   = datetime.date(2025, 12, 31)
date_span  = (end_date - start_date).days          # ≈ 730 日

# ---------- ランダムユーティリティ ----------
rand_date       = lambda: (start_date + datetime.timedelta(days=random.randint(0, date_span))).isoformat()
rand_booking_id = lambda: f"B{random.randint(0, 9_999_999):07d}"
rand_flight_id  = lambda: f"{random.choice(airlines)}{random.randint(1, 999):03d}"
rand_route      = lambda: random.choice(routes)
def rand_fare(o, d):
    domestic = {"HND","NRT","ITM","CTS","KIX","OKA","FUK"}
    low, high = ((8_000, 35_000) if (o in domestic and d in domestic) else (30_000, 180_000))
    return round(random.randint(low, high) / 100) * 100   # 100 円単位

# ---------- 会員プール作成 ― 1〜5 回ずつリスト化してシャッフル ----------
user_pool = []
uid = 1
while len(user_pool) < N_ROWS:
    # 必要になるまで新しい会員を追加し続ける
    repeats = random.randint(MIN_PER_USER, MAX_PER_USER)
    user_pool.extend([uid] * repeats)
    uid += 1
random.shuffle(user_pool)                       # 各予約に誰が乗るかランダム化
print(f"★会員総数: {uid-1:,} 名  (プール長 = {len(user_pool):,})")

# ---------- 便×出発日ペアを作りながら予約行を生成 ----------
rows, used_pairs = [], set()

while len(rows) < N_ROWS:
    flight_id, flight_date = rand_flight_id(), rand_date()
    pair = (flight_id, flight_date)
    if pair in used_pairs:                      # 同じ便×日付は 1 回だけ
        continue
    used_pairs.add(pair)

    bookings_here = min(BOOKINGS_PER_PAIR, N_ROWS - len(rows))  # ← 500 件固定（端数調整あり）
    for _ in range(bookings_here):
        uid = user_pool.pop()                   # プールから 1 名取り出す
        origin, dest = rand_route()
        rows.append(Row(
            booking_id  = rand_booking_id(),
            user_id     = uid,
            flight_id   = flight_id,
            route_id    = f"{origin}-{dest}",
            flight_date = flight_date,
            fare_amount = rand_fare(origin, dest)
        ))

# ---------- DataFrame 化 ----------
df = spark.createDataFrame(rows)\
         .withColumn("fare_amount", col("fare_amount").cast("decimal(10,2)"))

# ---------- 動作確認 ----------
print("総行数:", df.count())   # 総行数を出力

print("会員ごとの搭乗回数上位10件（最大 5 回）")
df.groupBy("user_id").count().orderBy("count", ascending=False).show(10)

df.show(10, truncate=False)    # データを確認
df.printSchema()               # スキーマを表示

print("各便の日付ごとの予約人数（フライトIDごと）")
df.groupBy("flight_id", "flight_date").count() \
  .orderBy("flight_date", "count", ascending=[True, False]) \
  .show(truncate=False)

print("各ユーザーの購入回数分布（1〜5 回）")
df.groupBy("user_id").count() \
  .groupBy("count").count() \
  .orderBy("count", ascending=False) \
  .show()

# # ---------- Delta テーブル保存 ----------
# df.write.format("delta")\
#   .option("comment", "フライト予約履歴")\
#   .mode("overwrite")\
#   .saveAsTable(f"{MY_CATALOG}.{MY_SCHEMA}.bz_flight_booking_2")

# ---------- CSV出力 ----------
out_path = f"/Volumes/{MY_CATALOG}/{MY_SCHEMA}/{MY_VOLUME}/flight_booking/flight_booking.csv"
df.coalesce(1).toPandas().to_csv(out_path, index=False)
print(out_path)

In [0]:
print("フライト予約履歴CSVファイルを作成しました！")